In [6]:
from selenium import webdriver

In [7]:
driver=webdriver.Chrome(executable_path="C:/Users/nango/selenium/chromedriver")

In [8]:
from lxml import html

In [9]:
import time
def find_page(link):
    listings=[]
    elements=link.xpath('//div[@class="iva-item-content-m2FiN"]/div[@class="iva-item-body-NPl6W"]')
    for element in elements:
        prop_name=element.xpath('./div[@class="iva-item-titleStep-2bjuh"]/a/@title')[0]
        price=element.xpath('./div[@class="iva-item-priceStep-2qRpg"]/span/span/meta[@itemprop="price"]/@content')[0]
        region=element.xpath('./div[@class="iva-item-developmentNameStep-1hr7p"]/div/span/span/text()')
        if len(region)>0:
            region=region[0]
        else:
            region=None
        listings.append({"prop_name":prop_name,"price":price,"region":region})
    return listings

In [10]:
all_listings=[]
for i in range(1,100+1):
    driver.get(f"https://www.avito.ru/rossiya/doma_dachi_kottedzhi?cd=1&p={i}")
    time.sleep(10)
    html_page=driver.page_source
    tree=html.fromstring(html_page)
    listings=find_page(tree)
    if len(listings)>0:
        all_listings.extend(listings)
        time.sleep(10)
    else:
        print(i)
        break

84


In [11]:
# Second round to overcome 504 Gateway Error
for i in range(84,100+1):
    driver.get(f"https://www.avito.ru/rossiya/doma_dachi_kottedzhi?cd=1&p={i}")
    time.sleep(10)
    html_page=driver.page_source
    tree=html.fromstring(html_page)
    listings=find_page(tree)
    if len(listings)>0:
        all_listings.extend(listings)
        time.sleep(10)
    else:
        print(i)
        break

In [1]:
import pandas as pd

In [13]:
df_apartments=pd.DataFrame(all_listings)
df_apartments.to_csv('all_houses.csv',index=False,encoding='utf-8-sig')

In [15]:
driver.close()

In [2]:
df=pd.read_csv("all_houses.csv")

In [20]:
import re
df['house_area']=df.prop_name.apply(lambda x: re.findall(r"\s(\d.*)\sм²",x))

In [21]:
df['house_area_sqm']=df.house_area.apply(lambda x: float(x[0].replace(",",".")) if len(x)>0 else None)

In [39]:
df['plot_area_cot']=df.prop_name.apply(lambda x: re.findall(r'([0-9,]*?)\sсот.',x))

In [40]:
df['plot_area_cot']=df.plot_area_cot.apply(lambda x: float(x[0].replace(",",".")) if len(x)>0 else None)

In [46]:
def quick_convert(x):
    try:
        return int(x)
    except:
        return None

In [47]:
df["price"]=df.price.apply(lambda x: quick_convert(x))

In [49]:
new_df=df[['region','price','house_area_sqm','plot_area_cot']]

In [50]:
new_df.to_csv("all_houses_final.csv",index=False,encoding='utf-8-sig')